In [ ]:
!pip install requests
import requests as req
import pandas as pd 
from datetime import datetime, timedelta, timezone
import sqlalchemy
import sqlite3

In [116]:
DATABASE_LOCATION = "sqlite:///COVID_PR_Completo.sqlite"

In [112]:
# Date
today = datetime.today()
yesterday = today - timedelta(days=1,hours=3) #to do: UTC
yyyymm = str(yesterday.year)+"-"+str(yesterday.month)
ddmm = str(yesterday.day)+"_"+str(yesterday.month)

date = yesterday.strftime("%x")

#start_date = today - timedelta(days=90)
#delta = timedelta(days=1)

#while start_date <= today:
#    run script
#    start_date += delta

In [ ]:
# Extract CSV

if __name__ == "__main__":

  # Path
  informe = ['informe_epidemiologico','INFORME_EPIDEMIOLOGICO']
  geral = ['geral','GERAL','Geral']

  for i in informe:
      for g in geral: 
        try:
          path = f'https://www.saude.pr.gov.br/sites/default/arquivos_restritos/files/documento/{yyyymm}/{i}_{ddmm}_{g}.csv'
  # DataFrame
          df = pd.read_csv(path,sep=';')
        except Exception:
          pass

  # Index
  df["Extraido_em"] = date
  df.set_index("Extraido_em", inplace=True)

  data_PR = pd.DataFrame(df,columns=["IBGE_RES_PR",
                                     "IBGE_ATEND_PR",
                                     "SEXO",
                                     "IDADE_ORIGINAL",
                                     "MUN_RESIDENCIA",
                                     "MUN_ATENDIMENTO",
                                     "LABORATORIO",
                                     "DATA_DIAGNOSTICO",
                                     "DATA_CONFIRMACAO_DIVULGACAO",
                                     "DATA_INICIO_SINTOMAS",
                                     "OBITO",
                                     "DATA_OBITO",
                                     "DATA_OBITO_DIVULGACAO",
                                     "STATUS",
                                     "DATA_RECUPERADO_DIVULGACAO",
                                     "FONTE_DADO_RECUPERADO"])
  print(data_PR)

In [114]:
# Define validation function

def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if df is empty
    if data_PR.empty:
        print("No data downloaded")
        return False

#    Check for duplicates
#    if pd.Series(data_PR[""],data_PR.index).is_unique:
#        pass
#    else:
#        raise Exception("Duplicated primary key")

In [115]:
# VaLidate
if check_if_valid_data(data_PR):
    print("Valid data")

In [119]:
# Load
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('COVID_PR_Completo.db')
cursor = conn.cursor()

# Define tables - Insert date index
sql_query = """
CREATE TABLE IF NOT EXISTS COVID_PR_Completo( 
  IBGE_RES_PR VARCHAR(15),
  IBGE_ATEND_PR VARCHAR(15),
  SEXO VARCHAR(5),
  IDADE_ORIGINAL INTEGER,
  MUN_RESIDENCIA VARCHAR(30),
  MUN_ATENDIMENTO VARCHAR(30),
  LABORATORIO VARCHAR(50),
  DATA_DIAGNOSTICO TEXT,
  DATA_CONFIRMACAO_DIVULGACAO TEXT,
  DATA_INICIO_SINTOMAS TEXT,
  OBITO VARCHAR(5),
  DATA_OBITO TEXT,
  DATA_OBITO_DIVULGACAO TEXT,
  STATUS VARCHAR(30),
  DATA_RECUPERADO_DIVULGACAO TEXT,
  FONTE_DADO_RECUPERADO VARCHAR(50)
)
"""
#CONSTRAINT primary_key_constraint PRIMARY KEY () 

cursor.execute(sql_query)
print("Open database successfully")

Open database successfully


In [120]:
# Insert data from dataframe to sql database
try: 
    data_PR.to_sql("COVID_PR_Completo", engine, index=True, if_exists='append')
except:
    print("Data already exists in the database")

conn.close()
print("Close database successfully")

Close database successfully
